# Residual Block in three ways

So TensorFlow2.0 is pretty neat, you can write code in not 1, not 2 but 3 ways, with each being pretty darn fun.

Let's see how to write a simple Residual block in three ways in TensorFlow2.0

In [48]:
!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import numpy as np

Let's have some basic data to test on!

In [0]:
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.fashion_mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)
eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)

In [60]:
dataset = tfds.load('fashion_mnist')
def prep(datapoint):
  image = datapoint['image']
  label = datapoint['label']
  image /= 255
  image = tf.cast(image, tf.float32)
  # image = image[None, ...]
  return image, label

train = dataset['train'].map(prep).batch(32)
test = dataset['test'].map(prep)

W0720 01:38:16.061637 140064409622400 dataset_builder.py:397] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


## Method 1 : Sequential

The reason I prefer to use Reflection Padding is because of [this](https://twitter.com/karpathy/status/720622989289644033?lang=en) tweet by Andrej Karpathy.


In [0]:
class ReflectPad(tf.keras.layers.Layer):
  def __init__(self):
    super(ReflectPad, self).__init__()
    self.padding = [[0, 0], [1, 1], [1, 1], [0, 0]]

  def call(self, x):
    return tf.pad(x, self.padding, mode = 'REFLECT')

def residual_block(filters=256):
  x = tf.keras.Input(shape=[None, None, filters])

  block = tf.keras.Sequential([
  ReflectPad(),
  tf.keras.layers.Conv2D(filters, kernel_size=3,
                                    use_bias=False),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  ReflectPad(),
  tf.keras.layers.Conv2D(filters, kernel_size=3,
                                    use_bias=False),
  tf.keras.layers.BatchNormalization()
  ])

  output = x + block(x)
  
  model = tf.keras.Model(inputs=x, outputs=output)
  return model

That's it.
This contains two convolutional layers, along with BatchNormalisation. You can customize it a lot more.

## Method 2 : Functional

In [0]:
class ResidualBlock(tf.keras.Model):
  def __init__(self, filters):
    input_data = tf.keras.Input(shape=[None, None, filters])
    data = input_data
    padding = [[0, 0], [1, 1], [1, 1], [0, 0]]

    x = tf.pad(data, padding, mode='REFLECT')
    x = tf.keras.layers.Conv2D(filters, kernel_size=3,
                                        use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.pad(x, padding, mode='REFLECT')
    x = tf.keras.layers.Conv2D(filters, kernel_size=3,
                                        use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    output = x + input_data

    super(ResidualBlock, self).__init__(
        inputs=input_data, outputs=output)

## Method 3: Sub-Classing

In [0]:
class ResidualBlock(tf.keras.Model):
  def __init__(self, filters):
    super(ResidualBlock, self).__init__()
    self.filters = filters
    self.padding = [[0, 0], [1, 1], [1, 1], [0, 0]]

    self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size=3,
                                        use_bias=False)
    self.bn1 = tf.keras.layers.BatchNormalization()

    self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size=3,
                                        use_bias=False)
    self.bn2 = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = tf.pad(input_tensor, self.padding, mode='REFLECT')
    x = self.conv1(x)
    x = self.bn1(x, training=training)
    x = tf.nn.relu(x)

    x = tf.pad(x, self.padding, mode='REFLECT')
    x = self.conv2(x)
    x = self.bn2(x, training=training)

    x += input_tensor
    return x